astro_101 placeholder for (course name)

# Actividad astro_101
Este cuaderno le ayudará a instalar todo lo que necesita para realizar la reducción y análisis de los datos de [...].

Asegúrese de leerlo bien.

# 1 Setup
Lo primero que debe realizar es descargar los archivos necesarios para la actividad. Para esto, debe:


1.   enlazar GD al colab
2.   descargar archivos a su GD


El primer paso que debe tomar es conectar su Google Drive (hagalo con el asociado a su correo mail.udp.cl). Para esto, en el lado izquierdo aprete el ícono que tiene forma de carpeta. En el menú que se abre, haga click en el ícono de Google Drive (carpeta con un símbolo parecido al del reciclaje) y de permiso para su conexión.

Alternativamente, ejecute el código siguiente (haciendo click sobre el icono play).

In [1]:
import os
from google.colab import drive

In [2]:
# conectar drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pwd

/content


In [4]:
#Cambiamos el directorio de trabajo al de Google Drive.
os.chdir('drive/MyDrive')

In [8]:
#!rm -rf astro_101

In [9]:
#Creamos un directorio donde vamos a guardar todas las rutinas y los archivos.
!mkdir astro_101
os.chdir('astro_101')

In [10]:
#Descargamos las rutinas desde el repositorio de Github.
!git clone https://github.com/ReddTea/udp_astro_101
#!git pull https://github.com/ReddTea/udp_astro_101

Cloning into 'udp_astro_101'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 19 (delta 3), reused 15 (delta 2), pack-reused 0
Receiving objects: 100% (19/19), 11.93 KiB | 1.19 MiB/s, done.
Resolving deltas: 100% (3/3), done.


# 2 TEST
Nos aseguraremos que las rutinas pueden correr sin problemas con el siguiente test. Para esto es necesario:


1.   Ubicarse en la carpeta de trabajo
2.   Instalar las librerias necesarias
3.   Correr los test



In [11]:
os.chdir('udp_astro_101')

In [12]:
!pwd

/content/drive/MyDrive/astro_101/astro_101/udp_astro_101


## 2.1 Install python libraries

In [13]:
%%capture
import numpy as np
try:
    import reddemcee
except ModuleNotFoundError:
    !pip3 install git+https://github.com/ReddTea/reddemcee.git
    import reddemcee

try:
    import astroemperor as emp
except ModuleNotFoundError:
    !pip3 install git+https://github.com/ReddTea/astroemperor.git
    import astroemperor as emp

## 2.2 Test reddemcee

In [15]:
def log_like(x, ivar):
    return -0.5 * np.sum(ivar * x ** 2)

def log_prior(x):
    return 0.0

ndim, nwalkers = 5, 100
ntemps = 5
ivar = 1. / np.random.rand(ndim)
p0 = list(np.random.randn(10, nwalkers, ndim))

In [16]:
# init sampler
sampler = reddemcee.PTSampler(nwalkers, ndim, log_like, log_prior,
                              ntemps=ntemps, adaptative=True, logl_args=[ivar])

In [17]:
s0 = sampler.run_mcmc(p0, 200, 2)

100%|██████████| 400/400 [00:04<00:00, 85.05it/s]


## 2.3 Test astroemperor

In [18]:
!pwd

/content/drive/MyDrive/astro_101/astro_101/udp_astro_101


In [24]:
import sys
sys.path.append('/content/drive/MyDrive/astro_101/astro_101/udp_astro_101')

In [25]:
sim = emp.Simulation()
sim.set_engine('reddemcee')
setup = np.array([2, 50, 100, 2])

sim.load_data('51Peg')  # Target folder name in /datafiles/
sim.plot_all['paper_mode'] = True

                                                                                
                   ~~ Simulation Successfully Initialized ~~                    
                                                                                


                         Reading data from 51peg.vels
                          




In [26]:
sim.run_auto(setup, k_start=1, k_end=1)

                                       Offset block added, OffsetBlock          


                                       Jitter block added, JitterBlock          


                                   Keplerian block added, KeplerianBlock 1      





                                ~~ Setup Info ~~                                

Current Engine is            reddemcee 0.6

Number of cores is           2

Save location is             datalogs/51Peg/run_2/k1

Dynamical Criteria is        None

Posterior fit method is      Gaussian Mixtures

Limits constrain method is   sigma

Model Selection method is    BIC


                           ~~ Automatically Saving ~~                           


Logger       : ✔

Samples      : ✘

Posteriors   : ✔

Likelihoods  : ✔


Plots: Posteriors           : ✔

Plots: Keplerian Model      : ✔

Plots: Gaussian Mixture     : ✔

Plots: Parameter Histograms : ✔



                               ~~ Pre-Run Info ~~                               



Paramete


100%|██████████| 9/9 [00:00<00:00, 17.29it/s]




                                 ~~ Best Fit ~~                                 



Parameter           Posterior                 Value (max)    Value (mean)    Sigma  Limits
------------------  ----------------------  -------------  --------------  -------  ---------------------------
Period 1            ~𝓝 (1081.085, 457.835)       1105.7          1081.09   457.835  [1.500000e+00 2.187042e+03]
Amplitude 1         ~𝓝 (11.799, 6.891)             17.195          11.799    6.891  [ 0.    79.833]
Phase 1             ~𝓝 (2.841, 1.672)               0.365           2.841    1.672  [0.    6.283]
Eccentricity 1      ~𝓝 (0.094, 0.069)               0.051           0.094    0.069  [0. 1.]
Longitude 1         ~𝓝 (2.643, 1.66)                4.593           2.643    1.66   [0.    6.283]

------------------  ------------------  -------------  -------------  -------  -------------
Semi-Major Axis 1   ~𝓝 (2.012, 0.618)           2.093          2.012    0.618  [   0. 1000.]
Minimum Mass 1      ~𝓝 



100%|██████████| 4/4 [00:30<00:00,  7.52s/it]

100%|██████████| 2/2 [00:06<00:00,  3.49s/it]




                            Plotting Histograms Plot                            


...



                           Plotting Gaussian Mixtures                           



100%|██████████| 7/7 [00:04<00:00,  1.54it/s]



Time RUN         :  00:01:25

Time POSTPROCESS :  00:00:00

Time CALCULATE GM:  00:00:00

Time Plot model      :  00:00:03

Time Plot posteriors :  00:00:37

Time Plot histograms :  00:00:07

Time Plot betas      :  00:00:00

Time Plot arviz      :  00:00:25

Time Plot GM         :  00:00:04
                                  Cleaning Run                                  



100%|██████████| 3/3 [00:00<00:00, 130.09it/s]



BIC condition met!!

present BIC < past BIC - 5
2650.283 < inf - 5


                                                                                
                            ~~ Run came to an end ~~                            
                                                                                
                                       
                                        
